In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split,GridSearchCV
from tqdm import tqdm

from pathlib import Path
import warnings
warnings.filterwarnings(action = 'ignore')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
C:\Users\user\Anaconda3\lib\site-packages\dask\config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\user\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\user\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyy

In [2]:
train = pd.read_csv('C:/Users/user/Desktop/open/train.csv')
test = pd.read_csv('C:/Users/user/Desktop/open/test.csv')
submission = pd.read_csv('C:/Users/user/Desktop/open/sample_submission.csv')

In [3]:
print(train.shape)
print(test.shape)

(26457, 20)
(10000, 19)


# FEATURE_SET_1

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   gender             26457 non-null  int32   
 1   car                26457 non-null  int32   
 2   reality            26457 non-null  int32   
 3   child_num          26457 non-null  int64   
 4   income_total       26457 non-null  int32   
 5   income_type        26457 non-null  float64 
 6   edu_type           26457 non-null  int32   
 7   family_type        26457 non-null  int32   
 8   house_type         26457 non-null  int32   
 9   DAYS_BIRTH         26457 non-null  int64   
 10  DAYS_EMPLOYED      26457 non-null  int64   
 11  FLAG_MOBIL         26457 non-null  int64   
 12  phone              26457 non-null  int64   
 13  email              26457 non-null  int64   
 14  occyp_type         26457 non-null  int32   
 15  begin_month        26457 non-null  int32   
 16  cred

In [10]:
for data in [train, test]:
    
    ## 변수 변환
    data['birth_plus'] = data.DAYS_BIRTH*(-1)/365
    
    ## 라벨 인코딩
    data['gender']=LabelEncoder().fit_transform(data.gender)
    data['car']=LabelEncoder().fit_transform(data.car)
    data['reality']=LabelEncoder().fit_transform(data.reality)
    data['income_type']=LabelEncoder().fit_transform(data.income_type)
    data['edu_type']=LabelEncoder().fit_transform(data.edu_type)
    data['family_type']=LabelEncoder().fit_transform(data.family_type)
    data['house_type']=LabelEncoder().fit_transform(data.house_type)
    data.fillna('NAN',inplace=True)
    data['occyp_type']=LabelEncoder().fit_transform(data.occyp_type)
    
    ## type 변환
    data.income_total=data.income_total.astype(int)
    data.family_size=data.family_size.astype(int)
    data.begin_month=data.begin_month.astype(int)
    data.birth_plus=data.birth_plus.astype(int)

    ## 변수 제거
    data.drop(['index','family_size','work_phone'],axis=1,inplace=True)
    
    ## 파생 변수 생성
    data['income_weight']=data.income_total**2
    data['income_age']=data.income_total*train.DAYS_BIRTH
    data['income_emp']=data.income_total*train.DAYS_EMPLOYED
    ## 파생 변수 생성
    data.DAYS_BIRTH=data.DAYS_BIRTH**6
    data.income_type=1/(data.income_type)
    data.income_weight=1/(data.income_weight)**3
    data['DAYS_BIRTH_weight']=1/(data.DAYS_BIRTH)**2

## catboost는 categorical로 두면 안 됨
cat_credit=train.credit.astype('int').copy()
train.credit=train.credit.astype('category')

## feature_set 1 저장
ftr1 = train.drop('credit', axis=1).values
target1 = train['credit'].values
tst_ar1 = test.values # 행단위 list
n_class = 3

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             10000 non-null  int32  
 1   car                10000 non-null  int32  
 2   reality            10000 non-null  int32  
 3   child_num          10000 non-null  int64  
 4   income_total       10000 non-null  int32  
 5   income_type        10000 non-null  float64
 6   edu_type           10000 non-null  int32  
 7   family_type        10000 non-null  int32  
 8   house_type         10000 non-null  int32  
 9   DAYS_BIRTH         10000 non-null  int64  
 10  DAYS_EMPLOYED      10000 non-null  int64  
 11  FLAG_MOBIL         10000 non-null  int64  
 12  phone              10000 non-null  int64  
 13  email              10000 non-null  int64  
 14  occyp_type         10000 non-null  int32  
 15  begin_month        10000 non-null  int32  
 16  birth_plus         1000

In [11]:
#FEATURE_SET2

In [ ]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

## 365 변환
def days_to_plus(x):
    return (x*-1)/365

## 업무 시작일 변환
def plus(x):
    if x<0:
        return x*(-1)
    else:
        return 0

    
## 변수 변환 (업무 시작일과 나이)
for i in [trn, tst]:
    i['birth_plus'] = i.DAYS_BIRTH.map(days_to_plus)
    i['employed_plus'] = i.DAYS_EMPLOYED.map(days_to_plus)


## 라벨 인코딩
index_col = 'index'
target_col = 'credit'

cat_cols = [x for x in trn.columns if trn[x].dtype == 'object']
float_cols = [x for x in trn.columns.drop('credit') if trn[x].dtype == 'float64']
num_cols = [x for x in trn.columns if x not in cat_cols + [target_col]]
feature_cols = num_cols + cat_cols

lbe = LabelEncoder()
for i in cat_cols:
    trn[i] = lbe.fit_transform(trn[i].astype(str))
    tst[i] = lbe.transform(tst[i].astype(str))
    
for i in float_cols:
    trn[i] = trn[i].astype('int')
    tst[i] = tst[i].astype('int')
    

## 인덱스 설정 및 변수 제거
for i in [trn, tst]:
    i.set_index('index', inplace = True)
    i.drop(['family_size', 'employed_plus'], axis = 1, inplace = True)


## 파생변수 생성
for i in [trn,tst]:
    # 연봉 제곱 변수 제거
    i['income_total_2'] = i.income_total**2

    # DAYS_BIRTH 제곱 변수 추가
    i['DAYS_BIRTH**2'] = i.DAYS_BIRTH**2

    # income * days_birth
    i['income_age'] = i.income_total*i.DAYS_BIRTH

    # income * DAYS_EMPLOYED
    i['income_emp'] = i.income_total*i.DAYS_EMPLOYED


## 데이터 재표현 (비선형 변환)
for i in [trn, tst]:
    i['DAYS_BIRTH**2'] = np.log1p(i['DAYS_BIRTH**2'])
    i['income_total_2'] = i.income_total_2**2
    i['income_emp'] = i.income_emp**2
    i['income_age'] = i.income_age**3


## 변수 타입 전환
for i in [trn, tst]:
    i['income_total'] = i.income_total.astype(int)
    i['DAYS_BIRTH**2'] = i['DAYS_BIRTH**2'].astype(int)
trn.credit = trn.credit.astype('category')


## feature_set 2 저장
ftr2 = trn.drop('credit', axis=1).values
target2 = trn['credit'].values
tst_ar2 = tst.values
n_class = 3

In [ ]:
# LGB_1

In [ ]:
n_fold = 17
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr1.shape)

lgb_p_val1 = np.zeros((ftr1.shape[0], n_class))
lgb_p_tst1 = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    lgb_clf = LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 50)

    lgb_clf.fit(ftr1[i_trn], target1[i_trn],
            eval_set=[(ftr1[i_val], target1[i_val])])

    lgb_p_val1[i_val, :] = lgb_clf.predict_proba(ftr1[i_val])
    lgb_p_tst1 += lgb_clf.predict_proba(tst_ar1) / n_fold
    
print(f'{log_loss(target1, lgb_p_val1)}')
print(f'{confusion_matrix(target1, np.argmax(lgb_p_val1, axis=1))}%')

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/koko4/Desktop/credit/')
val_dir = Path('C:/Users/koko4/Desktop/credit/')

algo_name = 'lgb'
feature_name = '2021_j'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
# LGB_2

In [ ]:
n_fold = 17
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr1.shape)

lgb_p_val2 = np.zeros((ftr1.shape[0], n_class))
lgb_p_tst2 = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    lgb_clf = lgb.LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=175, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 0)

    lgb_clf.fit(ftr1[i_trn], target1[i_trn],
            eval_set=[(ftr1[i_val], target1[i_val])])

    lgb_p_val2[i_val, :] = lgb_clf.predict_proba(ftr1[i_val])
    lgb_p_tst2 += lgb_clf.predict_proba(tst_ar1) / n_fold
    
print(f'{log_loss(target1, lgb_p_val2)}')
print(confusion_matrix(target1, np.argmax(lgb_p_val2, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'lgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
#HGBM

In [ ]:
# 그리드 서치
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='accuracy',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=8, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred



In [ ]:
# 데이터 분할
X_train,X_val,y_train,y_val=train_test_split(ftr1,target1,test_size=0.25,random_state=40)


# 모델 객체 생성
model = HistGradientBoostingClassifier()


paramgrid = {'max_iter' : [1000],
            'learning_rate' : [0.08, 0.09, 0.1, 0.11],
             'max_depth' : [15,17,18,20,22,24,26, 28, 30],
             'max_leaf_nodes' : [26,28,30, 31, 32, 33, 34],
             'random_state' : [2021]
            }


model, pred = algorithm_pipeline(X_train, X_val, y_train, y_val, model, 
                                 paramgrid, cv=10, scoring_fit='accuracy')


print(model.best_score_)
print(model.best_params_)

In [ ]:
n_fold = 10
print(ftr1.shape)
print(tst_ar1.shape)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=40)
 
hgb_p_val = np.zeros((ftr1.shape[0], n_class))
hgb_p_tst = np.zeros((tst_ar1.shape[0], n_class))
 
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    hgb_clf = HistGradientBoostingClassifier(max_iter=1000,
                                             learning_rate = 0.08,
                                                verbose=True,
                                             max_leaf_nodes = 33,
                                             max_depth = 18,
                                                random_state=2021
                                                )
    hgb_clf.fit(ftr1[i_trn], target1[i_trn])
    hgb_p_val[i_val, :] = hgb_clf.predict_proba(ftr1[i_val])
    hgb_p_tst += hgb_clf.predict_proba(tst_ar1) / n_fold
    
print(f'{log_loss(target1, hgb_p_val)}')
print(confusion_matrix(target1, np.argmax(hgb_p_val, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'hgbm'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, hgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, hgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
# xgboost

In [ ]:
n_fold = 17

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr1.shape)

xgb_p_val = np.zeros((ftr1.shape[0], n_class))
xgb_p_tst = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    xgb_clf = xgb.XGBClassifier(max_depth=10,
                                colsample_bytree=0.5,
                                colsample_bylevel=0.8,
                                colsample_bynode=0.5,
                                min_child_weight=0.5,
                                gamma=0,
                                max_delta_step=0,
                                subsample=0.9,
                                reg_alpha=0.09,
                                reg_lambda=0.6,
                                n_estimators=50,
                               random_state = 2021)

    xgb_clf.fit(ftr1[i_trn], target1[i_trn],
            eval_set=[(ftr1[i_val], target1[i_val])])
    
    xgb_p_val[i_val, :] = xgb_clf.predict_proba(ftr1[i_val])
    xgb_p_tst += xgb_clf.predict_proba(tst_ar1) / n_fold


print(f'{log_loss(target1, xgb_p_val)}')
print(confusion_matrix(target1, np.argmax(xgb_p_val, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'xgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, xgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, xgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
#catboost

In [ ]:
n_fold = 10
target1 = cat_credit.values

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2020)
 
print(ftr1.shape)
 
cb_p_val = np.zeros((ftr1.shape[0], n_class))
cb_p_tst = np.zeros((tst_ar1.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr1, target1), 1):
    print(f'training model for CV #{i}')
    cb_clf = CatBoostClassifier(depth=8,
                                iterations=2000,
                                rsm=0.7,
                                ignored_features=5,
                                learning_rate=0.04,
                                l2_leaf_reg=1,
                               random_state = 2021)
    
    cb_clf.fit(ftr1[i_trn], target1[i_trn],
    eval_set=[(ftr1[i_val], target1[i_val])])
    
    cb_p_val[i_val, :] = cb_clf.predict_proba(ftr1[i_val])
    cb_p_tst += cb_clf.predict_proba(tst_ar1) / n_fold

    
print(f'{log_loss(target1, cb_p_val)}')
print(confusion_matrix(target1, np.argmax(cb_p_val, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'cat'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, cb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, cb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
#random forest

In [ ]:
n_fold = 10

print(ftr2.shape)
print(tst_ar2.shape)

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr2.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar2.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr2, target2), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 7000, 
                                random_state=2021,
                                max_features = 3,
                                max_depth = 28,
                                min_samples_split = 8,
                                n_jobs=4)
    
    rf_clf.fit(ftr2[i_trn], target2[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr2[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar2) / n_fold
    
print(f'{log_loss(target2, rf_p_val)}')
print(confusion_matrix(target2, np.argmax(rf_p_val, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'rf'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, rf_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, rf_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
#extra tree

In [ ]:
n_fold = 10

print(ftr2.shape)
print(tst_ar2.shape)

cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 42)

ext_p_val = np.zeros((ftr2.shape[0], n_class))
ext_p_tst = np.zeros((tst_ar2.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr2, target2), 1):
    print(f'training model for CV #{i}')
    ext_clf = ExtraTreesClassifier(
                                   max_depth = 30, 
                                   max_features = 4, 
                                   min_samples_split = 9, 
                                   n_estimators = 7000, 
                                   random_state = 2021,
                                  n_jobs = 4)

    ext_clf.fit(ftr2[i_trn], target2[i_trn])
    ext_p_val[i_val, :] = ext_clf.predict_proba(ftr2[i_val])
    ext_p_tst += ext_clf.predict_proba(tst_ar2) / n_fold

print(f'{log_loss(target2, ext_p_val)}')
print(confusion_matrix(target2, np.argmax(ext_p_val, axis=1)))

In [ ]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'extra'
feature_name = '0.711'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, ext_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, ext_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
#stacking

In [ ]:
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

model_names = ['rf_2021',
               'xgb_2021',
               'lgb_2021',
                 'extra_2021',
                'cat_2021',
               'lgb_2021_1',
                'hgbm_2021'
              ]


stk_trn = []
stk_tst = []
feature_names = []
for model in model_names:
    stk_trn.append(np.loadtxt(val_dir / f'{model}.val.csv', delimiter=','))
    stk_tst.append(np.loadtxt(tst_dir / f'{model}.tst.csv', delimiter=','))
    feature_names += [f'{model}_credit0', f'{model}_credit1', f'{model}_credit2']
    
stk_trn = np.hstack(stk_trn)
stk_tst = np.hstack(stk_tst)
feature_names

In [ ]:
stk_trn.shape
stk_tst.shape

In [ ]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

n_fold = 13
seed = 2021
n_class = 3

target=trn['credit'].values
y=target.copy()

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2021)

p_trn =np.zeros((stk_trn.shape[0], 3))
p_val = np.zeros((stk_trn.shape[0], 3))
p_tst = np.zeros((stk_tst.shape[0], 3))
for i, (i_trn, i_val) in enumerate(cv.split(stk_trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(boosting_type='gbdt',
                             max_depth=2,
                             num_leaves=10,
                             colsample_bytree=0.9,
                             n_estimators=110, 
                             min_child_samples=24,
                             subsample=0.5,
                             subsample_freq=1,
                             learning_rate=0.1,
                             random_state=1557,
                             verbose = 50)
    
    clf.fit(stk_trn[i_trn], y[i_trn],
            eval_set=[(stk_trn[i_val], y[i_val])],
            eval_metric='logloss',
            verbose=True)

    p_val[i_val, :] = clf.predict_proba(stk_trn[i_val])
    p_tst += clf.predict_proba(stk_tst) / n_fold
    
print(clf)

print(f'{log_loss(target, p_val)}')
print(f'{confusion_matrix(target, np.argmax(p_val, axis=1))}%')

In [ ]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub[sub.columns] = p_tst
sub.to_csv('stack_final.csv')